In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm
from model import InceptionResnetV2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = InceptionResnetV2()

In [4]:
model.to(device)

InceptionResnetV2(
  (model): Sequential(
    (0): Stem(
      (branch1): Sequential(
        (0): BasicConv2d(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
          (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
        )
        (1): BasicConv2d(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
          (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
        )
        (2): BasicConv2d(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
        )
      )
      (branch2_a): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
      (branch2_b): BasicConv2d(
        (conv): Conv2d(64, 96, kernel_size=(3, 3), stride=(2,

In [5]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((299, 299)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomPerspective(distortion_scale=0.2),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor()
])

val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((299, 299)),
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(root='./data/train', transform=train_transforms)
val_dataset = torchvision.datasets.ImageFolder(root='./data/val', transform=val_transforms)

In [6]:
def accuracy(outputs, labels):
    _, preds = torch.max(F.log_softmax(outputs, dim=1), dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [8]:
def evaluate(model, batch, criterion, optimizer):
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)

    prediction = model(inputs)
    loss = criterion(prediction, labels)
    acc = accuracy(prediction, labels)

    return loss, acc

In [9]:
def train(model, batch, criterion, optimizer):
    optimizer.zero_grad()
    
    loss, acc = evaluate(model, batch, criterion, optimizer)
    
    loss.backward()
    optimizer.step()
    return loss, acc

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [11]:
max_epochs = 100
min_loss = np.inf
cur_patience = 0
patience = 10

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0

    model.train()
    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader), leave=False):
        batch_loss, batch_acc = train(model, batch, criterion, optimizer)
        train_loss += batch_loss.item()
        train_acc += batch_acc
    
    model.eval()
    with torch.no_grad():
        for i, batch in tqdm(enumerate(val_loader), total=len(val_loader), leave=False):
            batch_loss, batch_acc = evaluate(model, batch, criterion, optimizer)
            val_loss += batch_loss.item()
            val_acc += batch_acc

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    
    print(f'Epoch: {epoch}')
    print(f'Train Loss: {train_loss:.4}, Train Acc: {train_acc:.3}')
    print(f'Validation Loss: {val_loss:.4}, Validation Acc: {val_acc:.3}')
    print(f'-' * 100)

    
    if val_loss < min_loss:
        cur_patience = 0
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break

Epoch: 1
Train Loss: 3.874, Train Acc: 0.397
Validation Loss: 1.671, Validation Acc: 0.567
----------------------------------------------------------------------------------------------------


Epoch: 2
Train Loss: 2.842, Train Acc: 0.497
Validation Loss: 2.243, Validation Acc: 0.471
----------------------------------------------------------------------------------------------------


Epoch: 3
Train Loss: 2.282, Train Acc: 0.508
Validation Loss: 9.025, Validation Acc: 0.479
----------------------------------------------------------------------------------------------------


Epoch: 4
Train Loss: 2.237, Train Acc: 0.504
Validation Loss: 1.055, Validation Acc: 0.663
----------------------------------------------------------------------------------------------------


Epoch: 5
Train Loss: 1.542, Train Acc: 0.617
Validation Loss: 12.87, Validation Acc: 0.561
----------------------------------------------------------------------------------------------------


...

Epoch: 30
Train Loss: 0.3263, Train Acc: 0.892
Validation Loss: 0.3006, Validation Acc: 0.923
----------------------------------------------------------------------------------------------------


Epoch: 31
Train Loss: 0.529, Train Acc: 0.862
Validation Loss: 0.4219, Validation Acc: 0.873
----------------------------------------------------------------------------------------------------


Epoch: 32
Train Loss: 0.4424, Train Acc: 0.872
Validation Loss: 0.2243, Validation Acc: 0.929
----------------------------------------------------------------------------------------------------


Epoch: 33
Train Loss: 0.3186, Train Acc: 0.895
Validation Loss: 0.509, Validation Acc: 0.844
----------------------------------------------------------------------------------------------------


Epoch: 34
Train Loss: 0.8505, Train Acc: 0.809
Validation Loss: 0.9736, Validation Acc: 0.798
----------------------------------------------------------------------------------------------------


In [12]:
torch.save(best_model, 'model.pt')